In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import cv2
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import os
from os import listdir
from os.path import isfile, join
import random
import numpy as np

seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [2]:
from PIL import Image


class train_dataset(Dataset):

    def __init__(self, dataset_path, transform=None, training=True):
        self.classes = [2, 4, 5, 6, 7]
        self.dataset_path = dataset_path
        self.transform = transform
        self.img_path = []
        self.labels = []
        self.training = training

        for l in self.classes:
            labelPath = join(dataset_path, str(l))
            file_names = [
                f for f in listdir(labelPath) if isfile(join(labelPath, f))
            ]

            for file_name in file_names:
                self.img_path.append([join(labelPath, file_name), file_name])
                self.labels.append(self.classes.index(l))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        path, name = self.img_path[index]
        image = Image.open(path)
        if self.training:
            image = image.rotate(180)
        if self.transform:
            image = self.transform(image)
        return image, self.labels[index]
      
class test_dataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.dataset_path = dataset_path
        self.transform = transform
        self.img_path = []
        file_names = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]
        file_names.sort()
        for file_name in file_names:
            self.img_path.append([join(dataset_path, file_name), file_name])

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, index):
        path, name = self.img_path[index]
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        image = cv2.rotate(image, cv2.ROTATE_180)
        if self.transform:
            image = self.transform(image)
        return image, name


In [3]:
labels = [2, 4, 5, 6, 7]
train_data =  train_dataset('hw1/Training_data',
                         transform=transforms.Compose([transforms.Grayscale(), transforms.ToTensor()]))
train_loader = DataLoader(train_data, batch_size=512, shuffle=True)

valid_data = train_dataset('hw1/MNIST/valid',
                         transform=transforms.Compose([transforms.Grayscale(), transforms.ToTensor()]),
                         training=False)
valid_loader = DataLoader(valid_data, batch_size=512, shuffle=False)

# test_data = test_dataset('Testing_data1', transform=transforms.ToTensor())
# test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

In [4]:
class LeNet(nn.Module):
    def __init__(self, num_classes):
        super(LeNet, self).__init__()
        self.cnn = nn.Sequential(
          nn.Conv2d(1, 6, 5),
          nn.Dropout(0.2),
          nn.BatchNorm2d(6),
          nn.ReLU(True),
          nn.MaxPool2d(2, 2),
          nn.Conv2d(6, 16, 5),
          nn.BatchNorm2d(16),
          nn.Dropout(0.2),
          nn.ReLU(True),
          nn.MaxPool2d(2, 2),
        )
        self.fc = nn.Sequential(nn.Linear(4 * 4 * 16, 120), 
                                nn.Dropout(0.2),
                                nn.BatchNorm1d(120),
                                nn.ReLU(True), 
                                nn.Linear(120, 84),
                                nn.Dropout(0.2),
                                nn.BatchNorm1d(84),
                                nn.ReLU(True),
                                nn.Linear(84, num_classes))
    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x


In [5]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LeNet(len(labels)).to(device)

learning_rate = 0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
epochs = 10

In [14]:
from torchsummary import summary

summary(model, (1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 4, 4]            --
|    └─Conv2d: 2-1                       [-1, 6, 24, 24]           156
|    └─Dropout: 2-2                      [-1, 6, 24, 24]           --
|    └─BatchNorm2d: 2-3                  [-1, 6, 24, 24]           12
|    └─ReLU: 2-4                         [-1, 6, 24, 24]           --
|    └─MaxPool2d: 2-5                    [-1, 6, 12, 12]           --
|    └─Conv2d: 2-6                       [-1, 16, 8, 8]            2,416
|    └─BatchNorm2d: 2-7                  [-1, 16, 8, 8]            32
|    └─Dropout: 2-8                      [-1, 16, 8, 8]            --
|    └─ReLU: 2-9                         [-1, 16, 8, 8]            --
|    └─MaxPool2d: 2-10                   [-1, 16, 4, 4]            --
├─Sequential: 1-2                        [-1, 5]                   --
|    └─Linear: 2-11                      [-1, 120]                 30,840
|    └─

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 4, 4]            --
|    └─Conv2d: 2-1                       [-1, 6, 24, 24]           156
|    └─Dropout: 2-2                      [-1, 6, 24, 24]           --
|    └─BatchNorm2d: 2-3                  [-1, 6, 24, 24]           12
|    └─ReLU: 2-4                         [-1, 6, 24, 24]           --
|    └─MaxPool2d: 2-5                    [-1, 6, 12, 12]           --
|    └─Conv2d: 2-6                       [-1, 16, 8, 8]            2,416
|    └─BatchNorm2d: 2-7                  [-1, 16, 8, 8]            32
|    └─Dropout: 2-8                      [-1, 16, 8, 8]            --
|    └─ReLU: 2-9                         [-1, 16, 8, 8]            --
|    └─MaxPool2d: 2-10                   [-1, 16, 4, 4]            --
├─Sequential: 1-2                        [-1, 5]                   --
|    └─Linear: 2-11                      [-1, 120]                 30,840
|    └─

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
writer.add_graph(model,torch.rand(1,1,28,28).cuda())


In [23]:
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
exp_name = 'project2_mnist_with_both.txt'
f = open(exp_name, 'w')

for epoch in range(epochs):
    model.train()

    for i, (image, label) in tqdm(enumerate(train_loader)):
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        if (i+1) % 1000 == 0:
            print('Epoch: {} / {}  Loss: {}'.format(epoch+1, epochs, loss.item()))

        writer.add_scalar('Train Loss', loss.item(), epoch*len(train_loader)+i)

    model.eval()
    accuracy = 0.0
    total = 0
    with torch.no_grad():
        for image, label in valid_loader:
            image, label = image.to(device), label.to(device)
            outputs = model(image)
            predict = outputs.argmax(dim=-1)
            accuracy += (predict == label).sum().item()
            total += label.shape[0]

        print('Epoch {} / {}  Accuracy: {:.4f}%'.format(epoch+1, epochs, 100*(accuracy/total)))
        writer.add_scalar('Valid Accuracy', 100 * (accuracy / total), epoch)

        f.write('{}\n'.format(100 * (accuracy / total)))

f.close()


40it [00:03, 12.23it/s]


Epoch 1 / 10  Accuracy: 98.8348%


40it [00:03, 12.00it/s]


Epoch 2 / 10  Accuracy: 99.0392%


40it [00:03, 11.93it/s]


...

Epoch 8 / 10  Accuracy: 99.3459%


40it [00:03, 12.22it/s]


Epoch 9 / 10  Accuracy: 99.3868%


40it [00:03, 12.32it/s]


Epoch 10 / 10  Accuracy: 99.1006%


In [14]:
# with open('711183116.txt', 'w') as f:
#   for img, file_name in test_loader:
#     img = img.to(device)
#     output = model(img)
#     predict = output.argmax(dim=-1)
    
#     print('{} {}\n'.format(file_name[0].split('.')[0], labels[predict]))
#     f.write('{} {}\n'.format(file_name[0].split('.')[0], labels[predict]))